# VERISETİNİN OLUŞTURULMASI

Anladım, bu durumda açıklamanızı şöyle güncelleyebilirsiniz:

Bu Python kodu, beyin tümörlerine ait görüntüleri tek bir veri seti olarak düzenlemek için bir CSV dosyası oluşturuyor. os ve csv kütüphanelerini kullanarak, klasörlerdeki tümör türlerini döngü ile okuyup, her görüntünün dosya adını ve ait olduğu tümör türünü CSV dosyasına kaydediyoruz. Bu yöntemle, verilerimizi etkili bir biçimde sınıflandırarak tek bir veri setinde topluyoruz.

In [1]:
import os
import csv

# Dataset dizininin yolunu belirtin
dataset_path = './'

# CSV dosyasını oluştur
with open('tumor_dataset.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['filename', 'tumor_type'])  # Başlık satırı
    
    # Eğitim ve test veri setini işleyin
    for dataset_type in ['training', 'testing']:
        dataset_folder = os.path.join(dataset_path, dataset_type)
        print(dataset_folder)
        for tumor_type in os.listdir(dataset_folder):
            tumor_folder = os.path.join(dataset_folder, tumor_type)
            print(tumor_folder)
            for img_file in os.listdir(tumor_folder):
                # Her resmin dosya adı, tümör tipi ve veri grubu bilgilerini yaz
                csvwriter.writerow([dataset_folder+"/"+tumor_type+"/"+img_file, tumor_type])
                print(img_file)
    

./training
./training\glioma
Tr-glTr_0000.jpg
Tr-glTr_0001.jpg
Tr-glTr_0002.jpg
Tr-glTr_0003.jpg
Tr-glTr_0004.jpg
Tr-glTr_0005.jpg
Tr-glTr_0006.jpg
Tr-glTr_0007.jpg
Tr-glTr_0008.jpg
Tr-glTr_0009.jpg
Tr-gl_0010.jpg
Tr-gl_0011.jpg
Tr-gl_0012.jpg
Tr-gl_0013.jpg
Tr-gl_0014.jpg
Tr-gl_0015.jpg
Tr-gl_0016.jpg
Tr-gl_0017.jpg
Tr-gl_0018.jpg
Tr-gl_0019.jpg
Tr-gl_0020.jpg
Tr-gl_0021.jpg
Tr-gl_0022.jpg
Tr-gl_0023.jpg
Tr-gl_0024.jpg
Tr-gl_0025.jpg
Tr-gl_0026.jpg
Tr-gl_0027.jpg
Tr-gl_0028.jpg
Tr-gl_0029.jpg
Tr-gl_0030.jpg
Tr-gl_0031.jpg
Tr-gl_0032.jpg
Tr-gl_0033.jpg
Tr-gl_0034.jpg
Tr-gl_0035.jpg
Tr-gl_0036.jpg
Tr-gl_0037.jpg
Tr-gl_0038.jpg
Tr-gl_0039.jpg
Tr-gl_0040.jpg
Tr-gl_0041.jpg
Tr-gl_0042.jpg
Tr-gl_0043.jpg
Tr-gl_0044.jpg
Tr-gl_0045.jpg
Tr-gl_0046.jpg
Tr-gl_0047.jpg
Tr-gl_0048.jpg
Tr-gl_0049.jpg
Tr-gl_0050.jpg
Tr-gl_0051.jpg
Tr-gl_0052.jpg
Tr-gl_0053.jpg
Tr-gl_0054.jpg
Tr-gl_0055.jpg
Tr-gl_0056.jpg
Tr-gl_0057.jpg
Tr-gl_0058.jpg
Tr-gl_0059.jpg
Tr-gl_0060.jpg
Tr-gl_0061.jpg
Tr-gl_0062.jpg
Tr-gl_

# Beyin Tümörü Veri Setinin Sayısallaştırılması ve Güncellenmesi

Bu Python kodu, bir tümör türü veri setindeki kategorik verileri sayısal değerlere dönüştürmek amacıyla kullanılır. İlk olarak, pandas kütüphanesi kullanılarak veri seti tumor_dataset.csv dosyasından okunur ve bir DataFrame'e yüklenir. Kod, tümör türlerini (örneğin, 'glioma', 'menenjiom') sayısal değerlere eşlemek için her benzersiz tür için bir sayı atayan bir sözlük oluşturur. Bu sayısal değerler, yeni bir sütun olarak eklenir ve eski kategorik sütun veri setinden silinir. Güncellenmiş veri seti, updated_tumor_dataset.csv dosyasına kaydedilerek sayısal işleme uygun hale getirilmiş olur.

In [2]:
import pandas as pd  # Veri analizi için pandas kütüphanesi

# Mevcut veri setini oku
tumor_dataset = pd.read_csv('tumor_dataset.csv')  # 'tumor_dataset.csv' dosyasını oku ve bir DataFrame'e yükle

tumor_dataset  # Veri setini görüntüle (Jupyter Notebook'ta otomatik olarak çıktı verir)

# Tümör türlerini sayısal değerlere eşleyin
tumor_types = tumor_dataset['tumor_type'].unique()  # 'tumor_type' sütunundaki benzersiz tümör türlerini al
mapping = {tumor: index for index, tumor in enumerate(tumor_types)}  # Her tümör türüne bir sayısal değer atamak için bir eşleme sözlüğü oluştur

# Sayısal değerleri ekle
tumor_dataset['tumor_numeric'] = tumor_dataset['tumor_type'].map(mapping)  # 'tumor_numeric' adında yeni bir sütun ekleyerek 'tumor_type' değerlerini sayısal değerlere dönüştür

# Tümör türü sütununu sil
tumor_dataset.drop('tumor_type', axis=1, inplace=True)  # 'tumor_type' sütununu veri setinden sil

# Güncellenmiş veri setini kaydet
tumor_dataset.to_csv('updated_tumor_dataset.csv', index=False)  # Güncellenmiş veri setini 'updated_tumor_dataset.csv' olarak kaydet, dizin numaralarını kaydetme


# OpenCV Kütüphanesinin Python Ortamına Kurulumu

Bu komut, Python programlama dilinde görüntü işleme ve bilgisayarla görme uygulamaları geliştirmek için gereken OpenCV kütüphanesini kurar. opencv-python, temel görüntü işleme işlevlerinin yanı sıra, makine öğrenimi ve derin öğrenme projelerinde de yaygın olarak kullanılan güçlü bir araçtır. Bu kütüphane ile görüntüleri okuma, yazma, düzenleme ve analiz etme gibi işlemler kolaylıkla gerçekleştirilebilir. Kurulum tamamlandıktan sonra, OpenCV'nin sunduğu çeşitli fonksiyonlar ve sınıflar kullanılarak etkili görüntü işleme uygulamaları geliştirilebilir.

In [3]:
!pip install opencv-python



# Harris Köşe Tespiti ile Görüntü Analizi

Bu kod, bir veri setinden yüklenen görüntüler üzerinde Harris köşe tespiti algoritmasını uygulamak için kullanılmaktadır. İlk olarak, güncellenmiş tümör veri seti updated_tumor_dataset.csv dosyasından yüklenir ve her görüntü için belirtilen dosya yolu kullanılarak görüntüler okunur. Eğer bir görüntü yüklenemezse, hata mesajı verilir. Görüntüler gri tonlamaya dönüştürüldükten sonra Harris köşe tespiti uygulanır ve tespit edilen köşeler kırmızı renkle işaretlenir. Sonuçlar, belirtilen dizinde (harris_corners) kaydedilerek her görüntü için ayrı dosyalar oluşturulur. Bu işlem, görüntü analizi ve işleme süreçlerinde köşe tespiti için önemli bir adımdır.

In [4]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt

# Veri setini yükle
tumor_dataset = pd.read_csv('updated_tumor_dataset.csv')

# Sonuçları saklamak için bir dizin oluştur
output_dir = 'harris_corners'
os.makedirs(output_dir, exist_ok=True)

# Her görüntü üzerinde döngü başlat
for index, row in tumor_dataset.iterrows():
    # Dosya yolunu al
    img_path = row['filename']  # Tam yolu kullanın
    img = cv2.imread(img_path)

    # Eğer görüntü yüklenmediyse geç
    if img is None:
        print(f"Görüntü yüklenemedi: {img_path}")
        continue

    # Gri tonlamaya dönüştür
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Harris köşe tespiti uygula
    corners = cv2.cornerHarris(gray, 2, 3, 0.04)

    # Köşe tespiti sonuçlarını işaretle
    img[corners > 0.01 * corners.max()] = [0, 0, 255]  # Tespit edilen köşeleri kırmızı yap

    # Sonucu kaydet
    output_path = os.path.join(output_dir, f'harris_corners_{index}.jpg')
    cv2.imwrite(output_path, img)

   


# Harris Algoritması ile Kenar Tespiti: Görsellerden Graf Üretimi

Bu bölümde, görüntü işleme ve veri analizi için gerekli olan üç temel Python kütüphanesini yükleyeceğiz.

    OpenCV (opencv-python): Bilgisayarla görme uygulamaları için yaygın olarak kullanılan bir kütüphanedir. Görüntü işleme ve analiz işlemleri için güçlü araçlar sunar.

    NumPy: Sayısal hesaplamalar için temel bir kütüphanedir. Diziler ve matrisler gibi çok boyutlu veri yapıları ile çalışmamızı sağlar. Görüntü verilerini kolayca işlemek için gereklidir.

    Matplotlib: Veri görselleştirme için kullanılan bir kütüphanedir. Çizim, grafik ve diğer görsel gösterimler oluşturmak için geniş olanaklar sunar.

In [1]:
!pip install opencv-python numpy matplotlib


Bu Python kodu, Harris köşe tespit algoritması ile işaretlenmiş 7300 resmin analiz edilerek, her bir görüntüdeki köşe noktalarını ve kenar bilgilerini graf biçiminde temsil etmeyi amaçlamaktadır. Görüntülerdeki köşe noktaları, verinin daha iyi anlaşılmasını sağlamak ve daha ileri analizler için temel oluşturarak grafik yapısında düzenlenir. Bu yöntem, görüntü işleme ve bilgisayarla görü alanında önemli bir adım olarak kabul edilmektedir.

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Harris köşelerini tespit etme ve grafiği gösterme
def process_image(image_path):
    try:
        # Resmi yükle
        image = cv2.imread(image_path)
        if image is None:
            print(f"Resim yüklenemedi: {image_path}")
            return

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Harris köşeleri tespiti
        gray = np.float32(gray)
        corners = cv2.cornerHarris(gray, 2, 3, 0.04)

        # Köşelerin koordinatlarını al
        y_coords, x_coords = np.where(corners > 0.01 * corners.max())

        # Grafik oluşturma
        plt.figure(figsize=(8, 6))
        plt.scatter(x_coords, y_coords, color='red', marker='o', s=5)
        plt.title('Harris Köşeleri Tespiti')
        plt.xlim(0, gray.shape[1])
        plt.ylim(gray.shape[0], 0)
        plt.axis('off')

        # Grafiği kaydetme
        output_path = os.path.join('graphs', os.path.basename(image_path).replace('.png', '.jpg').replace('.jpeg', '.jpg').replace('.jpg', '.jpg'))
        plt.savefig(output_path, bbox_inches='tight', facecolor='white', dpi=300)
        plt.close()  # Grafiği kapat

    except Exception as e:
        print(f"Hata oluştu: {e}")

# Klasör yolu
folder_path = 'harris_corners'
output_folder = 'graphs'
os.makedirs(output_folder, exist_ok=True)  # Çıktı klasörünü oluştur

# Tüm resimleri işle
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Resim uzantılarını kontrol et
        image_path = os.path.join(folder_path, filename)
        print(f"İşleniyor: {image_path}")  # Hangi resmin işlendiğini göster
        process_image(image_path)

print("Tüm resimler işlendi ve grafikler kaydedildi.")


İşleniyor: harris_corners\harris_corners_0.jpg
İşleniyor: harris_corners\harris_corners_1.jpg
İşleniyor: harris_corners\harris_corners_10.jpg
İşleniyor: harris_corners\harris_corners_100.jpg
İşleniyor: harris_corners\harris_corners_1000.jpg
İşleniyor: harris_corners\harris_corners_1001.jpg
İşleniyor: harris_corners\harris_corners_1002.jpg
İşleniyor: harris_corners\harris_corners_1003.jpg
İşleniyor: harris_corners\harris_corners_1004.jpg
İşleniyor: harris_corners\harris_corners_1005.jpg
İşleniyor: harris_corners\harris_corners_1006.jpg
İşleniyor: harris_corners\harris_corners_1007.jpg
İşleniyor: harris_corners\harris_corners_1008.jpg
İşleniyor: harris_corners\harris_corners_1009.jpg
İşleniyor: harris_corners\harris_corners_101.jpg
İşleniyor: harris_corners\harris_corners_1010.jpg
İşleniyor: harris_corners\harris_corners_1011.jpg
İşleniyor: harris_corners\harris_corners_1012.jpg
İşleniyor: harris_corners\harris_corners_1013.jpg
İşleniyor: harris_corners\harris_corners_1014.jpg
İşleniyor: